<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/c_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузим данные
df = pd.read_csv('inheritance_dataset.csv')

# Определим числовые и категориальные признаки
numerical_features = ['estate', 'debts', 'wills', 'share_husband', 'share_wife',
                      'share_father', 'share_mother', 'share_sons', 'share_daughters']
categorical_features = ['husband', 'wives', 'father', 'mother', 'sons', 'daughters',
                        'brothers_m', 'sisters_m', 'grandfather', 'grandmother']
